In [1]:
import importlib
import mysocket as sock

importlib.reload(sock)

<module 'mysocket' from '/home/nghiaht7/data-engineer/public-datasystems-s20/inclass/03_20-first/mysocket.py'>

In [2]:
webconn = sock.makeConnection("personal.denison.edu", 80)

In [3]:
lines = ["GET / HTTP/1.1",
         "Host: personal.denison.edu",
         ""
        ]

In [4]:
for line in lines:
    sock.sendString(webconn, line)
    sock.sendCRLF(webconn)

In [5]:
s = sock.receiveTillClose(webconn)

In [6]:
s

'HTTP/1.1 200 OK\r\nDate: Thu, 21 Oct 2021 15:01:52 GMT\r\nServer: Apache/2.4.6 (CentOS)\r\nAccept-Ranges: bytes\r\nContent-Length: 464\r\nConnection: close\r\nContent-Type: text/html; charset=iso-8859-1\r\n\r\n<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2//EN">\n<HTML> \n<HEAD>    \n<TITLE>Personal Pages - Denison University</TITLE>\n</HEAD> <BODY> \n<p>&ldquo;Disclaimer: The views and opinions expressed in this service are strictly those of the information provider. The provider assumes full responsibility and liability for the content of this service. The contents of this service have neither been reviewed nor approved by Denison University.&rdquo;</p>\n</BODY> \n</HTML> \n'

In [7]:
webconn.close()

In [8]:
def getMailCode(serverline):
    endpos = serverline.index(" ")
    first = serverline[:endpos]
    num = int(first)
    return num

In [9]:
mailserver = sock.makeConnection("mail.denison.edu", 25)

Error connecting to ('mail.denison.edu', 25)


In [41]:
s1 = sock.receiveToCRLF(mailserver)
status = getMailCode(s1)
print(repr(s1), status)

'220 smtpint2.cc.denison.edu ESMTP Postfix\r\n' 220


In [10]:
prefixlines = [
    "HELO 140.141.92.5",
    "MAIL FROM:<bressoud@denison.edu>",
    "RCPT TO:<tcbressoud@gmail.com>"
]

headers = [
    'From: "Dr. B" <bressoud@denison.edu>',
    'To: "Tom" <tcbressoud@gmail.com>',
    'Date: Sun, 29 Mar 2020 12:00:00 -0500',
    'Subject: Test message',
    ''
]

body = """Hello Tom-
This is a test message with four header lines and
five lines in the message body.

Best, Dr. B
"""

for line in prefixlines:
    sock.sendString(mailserver, line)
    sock.sendCRLF(mailserver)
    statusline = sock.receiveToCRLF(mailserver)
    assert getMailCode(statusline) == 250
    
sock.sendString(mailserver, "DATA")
sock.sendCRLF(mailserver)
statusline = sock.receiveToCRLF(mailserver)
assert getMailCode(statusline) == 354

for headerline in headers:
    sock.sendString(mailserver, headerline)
    sock.sendCRLF(mailserver)

for bodyline in body.split('\n'):
    sock.sendString(mailserver, bodyline)
    sock.sendCRLF(mailserver)

sock.sendString(mailserver, ".")
sock.sendCRLF(mailserver)

statusline = sock.receiveToCRLF(mailserver)
print(repr(statusline))
assert getMailCode(statusline) == 250

AttributeError: 'NoneType' object has no attribute 'sendall'

In [ ]:
sock.sendString(mailserver, "QUIT")
sock.sendCRLF(mailserver)
statusline = sock.receiveToCRLF(mailserver)
assert getMailCode(statusline) == 221

In [ ]:
mailserver.close()